In [2]:
pip install -U imbalanced-learn


  Obtaining dependency information for imbalanced-learn from https://files.pythonhosted.org/packages/a3/9e/fbe60a768502af54563dcb59ca7856f5a8833b3ad5ada658922e1ab09b7f/imbalanced_learn-0.11.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/235.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/235.6 kB ? eta -:--:--
   ----- --------------------------------- 30.7/235.6 kB 660.6 kB/s eta 0:00:01
   -------------------- ------------------- 122.9/235.6 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------  235.5/235.6 kB 2.4 MB/s eta 0:00:01
   ---------------------------------------- 235.6/235.6 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.10.1
    Uninstalling imbalanced-learn-0.10.1:
      Successfully uninstalled imbalanced-learn-0.10.1


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
import lightgbm

In [4]:
df1=pd.read_csv('./Originations_Data.csv'
df1.head()

C:\Users\vinee\AppData\Local\Temp\ipykernel_9788\197864035.py:1: DtypeWarning: Columns (26,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df1=pd.read_csv(r"C:\Users\vinee\OneDrive\Desktop\Case study\Originations_Data.csv")


,Credit Score,First Payment Date,First Time Homebuyer Flag,Maturity Date,Metropolitan Statistical Area,Mortgage Insurance Percentage,Number of Units,Occupancy Status,Original Combined Loan-to-Value,Original Debt-to-Income Ratio,...,Original Loan Term,Number of Borrowers,Seller Name,Servicer Name,Super Conforming Flag,Pre-HARP Loan Sequence Number,Program Indicator,HARP Indicator,Property Valuation Method,Interest Only Indicator
0,654,201803,N,204802,NaN,0,1,P,77,35,...,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N
1,693,201803,N,203302,24340.0,0,1,P,80,41,...,180,2,Other sellers,Other servicers,NaN,NaN,9,NaN,1,N
2,757,201803,Y,204802,NaN,25,1,P,97,10,...,360,1,Other sellers,Other servicers,NaN,NaN,H,NaN,2,N
3,807,201803,N,203302,NaN,0,1,P,26,38,...,180,2,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N
4,812,201803,N,203302,19340.0,0,1,P,14,14,...,180,2,Other sellers,Other servicers,NaN,NaN,9,NaN,1,N


In [5]:
df2 = pd.read_csv('./Behaviour_Data.csv')
df2.head()

,Loan Sequence Number,Delinquency_Bucket 201802,Delinquency_Bucket 201803,Delinquency_Bucket 201804,Delinquency_Bucket 201805,Delinquency_Bucket 201806,Delinquency_Bucket 201807,Delinquency_Bucket 201808,Delinquency_Bucket 201809,Delinquency_Bucket 201810,...,Delinquency_Bucket 202107,Delinquency_Bucket 202108,Delinquency_Bucket 202109,Delinquency_Bucket 202110,Delinquency_Bucket 202111,Delinquency_Bucket 202112,Delinquency_Bucket 202201,Delinquency_Bucket 202202,Delinquency_Bucket 202203,LossEstimate
0,F18Q10000028,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,F18Q10000052,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,F18Q10000084,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,F18Q10000117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,F18Q10000140,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
merged_df = pd.merge(df1, df2, on='Loan Sequence Number', how='inner')

In [7]:
merged_df.isna().sum()

Credit Score                         0
First Payment Date                   0
First Time Homebuyer Flag            0
Maturity Date                        0
Metropolitan Statistical Area    13113
                                 ...  
Delinquency_Bucket 202112        73158
Delinquency_Bucket 202201        74811
Delinquency_Bucket 202202        76047
Delinquency_Bucket 202203        77097
LossEstimate                         0
Length: 82, dtype: int64

In [8]:
delinquency_columns = df2.columns[1:48]

In [9]:
merged_df['Total_Delinquency'] = merged_df[delinquency_columns].sum(axis=1)
merged_df['Total_Delinquency']
merged_df

,Credit Score,First Payment Date,First Time Homebuyer Flag,Maturity Date,Metropolitan Statistical Area,Mortgage Insurance Percentage,Number of Units,Occupancy Status,Original Combined Loan-to-Value,Original Debt-to-Income Ratio,...,Delinquency_Bucket 202108,Delinquency_Bucket 202109,Delinquency_Bucket 202110,Delinquency_Bucket 202111,Delinquency_Bucket 202112,Delinquency_Bucket 202201,Delinquency_Bucket 202202,Delinquency_Bucket 202203,LossEstimate,Total_Delinquency
0,654,201803,N,204802,NaN,0,1,P,77,35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
1,693,201803,N,203302,24340.0,0,1,P,80,41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2,757,201803,Y,204802,NaN,25,1,P,97,10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,807,201803,N,203302,NaN,0,1,P,26,38,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
4,812,201803,N,203302,19340.0,0,1,P,14,14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141727,806,202012,N,204011,11244.0,0,1,P,49,37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
141728,785,202012,N,204011,35004.0,0,1,P,63,20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
141729,730,202012,N,203511,15680.0,0,1,P,52,47,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
141730,691,202012,N,205011,35980.0,25,1,P,97,42,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
merged_df.isnull().sum()

Credit Score                         0
First Payment Date                   0
First Time Homebuyer Flag            0
Maturity Date                        0
Metropolitan Statistical Area    13113
                                 ...  
Delinquency_Bucket 202201        74811
Delinquency_Bucket 202202        76047
Delinquency_Bucket 202203        77097
LossEstimate                         0
Total_Delinquency                    0
Length: 83, dtype: int64

In [11]:
delinquent_count = merged_df[merged_df['Total_Delinquency'] == 0]

In [12]:
delinquent_count

,Credit Score,First Payment Date,First Time Homebuyer Flag,Maturity Date,Metropolitan Statistical Area,Mortgage Insurance Percentage,Number of Units,Occupancy Status,Original Combined Loan-to-Value,Original Debt-to-Income Ratio,...,Delinquency_Bucket 202108,Delinquency_Bucket 202109,Delinquency_Bucket 202110,Delinquency_Bucket 202111,Delinquency_Bucket 202112,Delinquency_Bucket 202201,Delinquency_Bucket 202202,Delinquency_Bucket 202203,LossEstimate,Total_Delinquency
0,654,201803,N,204802,NaN,0,1,P,77,35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
1,693,201803,N,203302,24340.0,0,1,P,80,41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2,757,201803,Y,204802,NaN,25,1,P,97,10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,807,201803,N,203302,NaN,0,1,P,26,38,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
5,661,201803,N,204802,41660.0,0,1,P,61,45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141727,806,202012,N,204011,11244.0,0,1,P,49,37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
141728,785,202012,N,204011,35004.0,0,1,P,63,20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
141729,730,202012,N,203511,15680.0,0,1,P,52,47,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
141730,691,202012,N,205011,35980.0,25,1,P,97,42,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
Jan_Mar_Deliquency = [
    'Delinquency_Bucket 202201', 'Delinquency_Bucket 202202', 'Delinquency_Bucket 202203'
]

In [14]:
delinquent_count[Jan_Mar_Deliquency]=delinquent_count[Jan_Mar_Deliquency].fillna(0)

C:\Users\vinee\AppData\Local\Temp\ipykernel_9788\690143549.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  delinquent_count[Jan_Mar_Deliquency]=delinquent_count[Jan_Mar_Deliquency].fillna(0)


In [15]:
delinquent_count[Jan_Mar_Deliquency].isna().sum()

Delinquency_Bucket 202201    0
Delinquency_Bucket 202202    0
Delinquency_Bucket 202203    0
dtype: int64

In [16]:
def label_rolled_forward_or_stable(row):
    Jan_Mar_Deliquency1 = row[Jan_Mar_Deliquency].tolist()
    # Convert the Jan_Mar_Deliquency values to integers
    Jan_Mar_Deliquency1 = [int(status) for status in Jan_Mar_Deliquency1]
    
    if any([status > 0 for status in Jan_Mar_Deliquency1]):
        return 'Rolled Forward'
    else:
        return 'Stable'

In [17]:
delinquent_count['Roll_Status'] = delinquent_count.apply(label_rolled_forward_or_stable, axis=1)

C:\Users\vinee\AppData\Local\Temp\ipykernel_9788\2684978217.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  delinquent_count['Roll_Status'] = delinquent_count.apply(label_rolled_forward_or_stable, axis=1)


In [18]:
print(delinquent_count[['Loan Sequence Number', 'Roll_Status']])

       Loan Sequence Number Roll_Status
0              F18Q10000028      Stable
1              F18Q10000052      Stable
2              F18Q10000084      Stable
3              F18Q10000117      Stable
5              F18Q10000193      Stable
...                     ...         ...
141727         F20Q41275382      Stable
141728         F20Q41276537      Stable
141729         F20Q41280326      Stable
141730         F20Q41280605      Stable
141731         F20Q41282323      Stable

[128465 rows x 2 columns]


In [19]:
delinquent_count['Roll_Status'].value_counts()

Roll_Status
Stable            128076
Rolled Forward       389
Name: count, dtype: int64

Taking the features that seemto be more relevant

In [20]:
final_data=delinquent_count[['Credit Score','First Time Homebuyer Flag','Number of Units',
'Occupancy Status','Original Combined Loan-to-Value','Original Debt-to-Income Ratio','Original UPB',
'Original Loan-to-Value','Property Type','Loan Purpose']]

Separating the Numerical and Categorical columns

In [21]:
Numeric_cols=[col for col in final_data.columns if final_data[col].dtype !='object']

Categirical_cols=[col for col in final_data.columns if final_data[col].dtype =='object']

In [22]:
final_data=pd.get_dummies(final_data,columns= Categirical_cols)
cols=final_data.columns # columnn list for preparing dataframe after scaling

In [23]:
from sklearn.preprocessing import MinMaxScaler 
scale=MinMaxScaler()
scale.fit(final_data)
final_data=scale.transform(final_data)
final_data=pd.DataFrame(final_data,columns=cols)

In [24]:
final_data

,Credit Score,Number of Units,Original Combined Loan-to-Value,Original Debt-to-Income Ratio,Original UPB,Original Loan-to-Value,First Time Homebuyer Flag_N,First Time Homebuyer Flag_Y,Occupancy Status_I,Occupancy Status_P,Occupancy Status_S,Property Type_CO,Property Type_CP,Property Type_MH,Property Type_PU,Property Type_SF,Loan Purpose_C,Loan Purpose_N,Loan Purpose_P
0,0.018382,0.000000,0.072435,0.034068,0.027877,0.072435,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,0.022479,0.000000,0.075453,0.040080,0.086490,0.075453,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,0.029202,0.000000,0.092555,0.009018,0.012152,0.092555,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,0.034454,0.000000,0.021127,0.037074,0.029307,0.021127,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,0.019118,0.000000,0.056338,0.044088,0.315940,0.056338,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128460,0.034349,0.000000,0.044266,0.036072,0.342387,0.044266,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
128461,0.032143,0.000000,0.058350,0.019038,0.315225,0.058350,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
128462,0.026366,0.000000,0.047284,0.046092,0.192280,0.047284,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
128463,0.022269,0.000000,0.092555,0.041082,0.165118,0.092555,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [25]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128465 entries, 0 to 128464
Data columns (total 19 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   Credit Score                     128465 non-null  float64
 1   Number of Units                  128465 non-null  float64
 2   Original Combined Loan-to-Value  128465 non-null  float64
 3   Original Debt-to-Income Ratio    128465 non-null  float64
 4   Original UPB                     128465 non-null  float64
 5   Original Loan-to-Value           128465 non-null  float64
 6   First Time Homebuyer Flag_N      128465 non-null  float64
 7   First Time Homebuyer Flag_Y      128465 non-null  float64
 8   Occupancy Status_I               128465 non-null  float64
 9   Occupancy Status_P               128465 non-null  float64
 10  Occupancy Status_S               128465 non-null  float64
 11  Property Type_CO                 128465 non-null  float64
 12  Pr

In [26]:
X = final_data
Y = delinquent_count['Roll_Status']

In [27]:
Y.value_counts()

Roll_Status
Stable            128076
Rolled Forward       389
Name: count, dtype: int64

#### As we can see the data sample is imbalanced , Hence we use oversampling to reduce the imbalance

In [28]:
from imblearn.over_sampling import RandomOverSampler

In [31]:
rus=RandomOverSampler(random_state=42)
X_r,Y_r=rus.fit_resample(X,Y)

In [32]:
Y_r.value_counts()

Roll_Status
Stable            128076
Rolled Forward    128076
Name: count, dtype: int64

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X_r, Y_r, test_size=0.4, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(153691, 19)
(102461, 19)
(153691,)
(102461,)


In [34]:
model = lightgbm.LGBMClassifier(random_state=42,n_estimators=2000)
model.fit(X_train, y_train)
prediction = model.predict(X_test)
df = pd.DataFrame(prediction)
df.value_counts()

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 76976, number of negative: 76715
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007685 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 766
[LightGBM] [Info] Number of data points in the train set: 153691, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500849 -> initscore=0.003396
[LightGBM] [Info] Start training from score 0.003396


Rolled Forward    51383
Stable            51078
Name: count, dtype: int64

In [35]:
metrics.confusion_matrix(prediction,y_test)

array([[51361,    22],
       [    0, 51078]], dtype=int64)

In [36]:
metrics.accuracy_score(prediction,y_test)

0.9997852841568987

#### Let's check for predictions on actual samples with the trained model¶

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(77079, 19)
(51386, 19)
(77079,)
(51386,)


In [38]:
prediction = model.predict(X_test)
df = pd.DataFrame(prediction)
df.value_counts()

Stable            51212
Rolled Forward      174
Name: count, dtype: int64

In [39]:
metrics.accuracy_score(prediction,y_test)

0.9997470128050442

In [40]:
y_test.value_counts()

Roll_Status
Stable            51225
Rolled Forward      161
Name: count, dtype: int64

In [41]:
metrics.confusion_matrix(prediction,y_test)

array([[  161,    13],
       [    0, 51212]], dtype=int64)